In [2]:
import requests
import pandas as pd
import io
import os

from joblib import Parallel, delayed
from PIL import Image

In [3]:
colors = ['red','green','blue','yellow']
DIR = "external_data/"
v18_url = 'http://v18.proteinatlas.org/images/'

In [4]:
imgList = pd.read_csv("HPAv18RBGY_wodpl.csv")

In [5]:
imgList.head()

,Id,Target
0,10580_1610_C1_1,13 25 0 2 21
1,10580_1610_C1_2,13 25 0 2 21
2,10580_1756_B1_1,13 25 0 2 21
3,10580_1756_B1_2,13 25 0 2 21
4,10580_1758_B1_1,13 25 0 2 21


In [6]:
len(imgList)

74606

In [7]:
def load_img(url):
    img = url.split('_')
    for color in colors:
        img_path = img[0] + '/' + "_".join(img[1:]) + "_" + color + ".jpg"
        img_name = url + "_" + color + ".png"
        img_url = v18_url + img_path
        
        r = requests.get(img_url, allow_redirects=True)
        pilImage = Image.open(io.BytesIO(r.content))
        pilImage = pilImage.resize((512, 512), Image.ANTIALIAS)
        pilImage.save(DIR + img_name, quality_val=95)
        #open(DIR + img_name, 'wb').write(r.content)

In [ ]:
num_cores = 8
Parallel(n_jobs=num_cores, prefer="threads")(delayed(load_img)(i) for i in imgList['Id'])

In [10]:
len(os.listdir(DIR)), len(imgList)*4 #should be equal

(298424, 298424)